# PGM Basics

A probabilistic graphical model in Compiled Knowledge (CK) is represented by an object of type `PGM`.

In [1]:
from ck.pgm import PGM

pgm = PGM()

## Random variables ##

Random variables are added to a PGM. Each random variable has a name and number of states. The states are indexed from zero.

A random variable in CK is represented by an object of type `RandomVariable`. A random variable of a PGM is created with a name and a number of states.

In [2]:
rain      = pgm.new_rv('rain', 2)       # 0=not raining  1=raining
sprinkler = pgm.new_rv('sprinkler', 2)  # 0=sprinkler off  1=sprinkler on
grass     = pgm.new_rv('grass', 3)      # 0=grass dry  1=grass damp  2=grass wet

The name of a random variable is access by its `name` property, and the number of states by `len`.

In [3]:
print('Random variable', rain.name, 'has', len(rain), 'states.')

Random variable rain has 2 states.


Every random variable has an index (`idx`), which is its location in the PGM array of random variables.

In [4]:
print(rain.idx, sprinkler.idx, grass.idx)

0 1 2


The array of PGM random variables is called `rvs`.

In [5]:
print(pgm.rvs[0], pgm.rvs[1], pgm.rvs[2])

rain sprinkler grass


A random variable can be treated as a tuple of indicators. Each indicator is of type `Indicator`. An indicator represent the proposition that a particular random variable is in a particular state.

For example, the indicator for 'sprinkler' = 'on' ...

In [6]:
sprinkler[1]

Indicator(rv_idx=1, state_idx=1)

An indicator in CK is a simple, hashable object. However, a PGM provides a way to render an indicator as a human-readable string.

In [7]:
pgm.indicator_str(sprinkler[1])

'sprinkler=1'

This works for multiple indicators too.

In [8]:
pgm.indicator_str(sprinkler[1], grass[2], rain[0])

'sprinkler=1, grass=2, rain=0'

## Instances ##

Once PGM random variables are defined, it is possible to iterate over all possible worlds. Each possible world is called an _instance_. An instance is an array of states indices, where the array is co-indexed with the array of random variables.

In [9]:
print(*pgm.rvs)

for inst in pgm.instances():
    print(inst)

rain sprinkler grass
(0, 0, 0)
(0, 0, 1)
(0, 0, 2)
(0, 1, 0)
(0, 1, 1)
(0, 1, 2)
(1, 0, 0)
(1, 0, 1)
(1, 0, 2)
(1, 1, 0)
(1, 1, 1)
(1, 1, 2)


We can also iterate over the instances as indicators.

In [10]:
for inst in pgm.instances_as_indicators():
    print(inst)

(Indicator(rv_idx=0, state_idx=0), Indicator(rv_idx=1, state_idx=0), Indicator(rv_idx=2, state_idx=0))
(Indicator(rv_idx=0, state_idx=0), Indicator(rv_idx=1, state_idx=0), Indicator(rv_idx=2, state_idx=1))
(Indicator(rv_idx=0, state_idx=0), Indicator(rv_idx=1, state_idx=0), Indicator(rv_idx=2, state_idx=2))
(Indicator(rv_idx=0, state_idx=0), Indicator(rv_idx=1, state_idx=1), Indicator(rv_idx=2, state_idx=0))
(Indicator(rv_idx=0, state_idx=0), Indicator(rv_idx=1, state_idx=1), Indicator(rv_idx=2, state_idx=1))
(Indicator(rv_idx=0, state_idx=0), Indicator(rv_idx=1, state_idx=1), Indicator(rv_idx=2, state_idx=2))
(Indicator(rv_idx=0, state_idx=1), Indicator(rv_idx=1, state_idx=0), Indicator(rv_idx=2, state_idx=0))
(Indicator(rv_idx=0, state_idx=1), Indicator(rv_idx=1, state_idx=0), Indicator(rv_idx=2, state_idx=1))
(Indicator(rv_idx=0, state_idx=1), Indicator(rv_idx=1, state_idx=0), Indicator(rv_idx=2, state_idx=2))
(Indicator(rv_idx=0, state_idx=1), Indicator(rv_idx=1, state_idx=1), Indi

We can pretty-print the instances using `indicator_str`.

In [11]:
for inst in pgm.instances_as_indicators():
    print(pgm.indicator_str(*inst))

rain=0, sprinkler=0, grass=0
rain=0, sprinkler=0, grass=1
rain=0, sprinkler=0, grass=2
rain=0, sprinkler=1, grass=0
rain=0, sprinkler=1, grass=1
rain=0, sprinkler=1, grass=2
rain=1, sprinkler=0, grass=0
rain=1, sprinkler=0, grass=1
rain=1, sprinkler=0, grass=2
rain=1, sprinkler=1, grass=0
rain=1, sprinkler=1, grass=1
rain=1, sprinkler=1, grass=2


## Factors ##

Factors are added to a PGM by creating objects of type `Factor` using method `PGM.new_factor`.

When a PGM is to represent a Bayesian network, each factor is a conditional probability table (CPT). By convention, the child random variable of a CPT as first in the list of a factor's random variables.

In [12]:
factor_g = pgm.new_factor(grass, rain, sprinkler)          # same as a CPT with child ‘grass’
factor_r = pgm.new_factor(rain)
factor_s = pgm.new_factor(sprinkler)

In general, a factor defines a real number for each possible combination of states of the factor's random variables. In CK, the function from the combination of states to real number is represented by an object of type `PotentialFunction`. There are many kinds of `PotentialFunction` in CK.

When a factor is first created it is supplied with a `ZeroPotentialFunction` which returns zero of every possible combination of states. This can easily be replaced with a `DensePotentialFunction` using `set_dense()`. A `DensePotentialFunction` is a (multidimensional) array of parameter values, one for each possible combination of states.

In [13]:
from ck.pgm import DensePotentialFunction  # included merely to show types

f_g: DensePotentialFunction = factor_g.set_dense()
f_r: DensePotentialFunction = factor_r.set_dense()
f_s: DensePotentialFunction = factor_s.set_dense()

f_r[0] = 0.8
f_r[1] = 0.2

f_s[0] = 0.9
f_s[1] = 0.1

f_g[0, 0, 0] = 0.90
f_g[0, 0, 1] = 0.01
f_g[0, 1, 0] = 0.02
f_g[0, 1, 1] = 0.01
# and so on...

Alternatively, the parameters of a dense potential function may be set all at once, using `set_flat(...)`.

In [14]:
f_r.set_flat(0.8, 0.2)
f_s.set_flat(0.9, 0.1)
f_g.set_flat(
    # not raining,  raining       # rain
    # off, on,      off,   on     # sprinkler
    0.90, 0.01,     0.02, 0.01,   # grass dry
    0.09, 0.01,     0.08, 0.04,   # grass damp
    0.01, 0.98,     0.90, 0.95,   # grass wet
)

# Here is an example of the set value
print(f_g[2, 1, 1])

0.95


## Weight of an instance ##

A PGM provides a weight for each instance. It is the value of the product of all factors for that instance, hence is the method `PGM.value_product`. Note that the instance is co-indexed with the PGM random variables.

In [15]:
pgm.value_product([1, 0, 1])  # rain=1 sprinkler=0 grass=1

0.014400000000000001

You can also calculate the weight of an instance by passing indicators instead. When using indicators, the order of the indicators is not important.

In [16]:

pgm.value_product_indicators(rain[1], sprinkler[0], grass[1])

0.014400000000000001

Here are the weights for all instances

In [17]:
for inst in pgm.instances_as_indicators():
    weight = pgm.value_product_indicators(*inst)
    print(pgm.indicator_str(*inst), 'has weight', weight)

rain=0, sprinkler=0, grass=0 has weight 0.6480000000000001
rain=0, sprinkler=0, grass=1 has weight 0.0648
rain=0, sprinkler=0, grass=2 has weight 0.007200000000000001
rain=0, sprinkler=1, grass=0 has weight 0.0008
rain=0, sprinkler=1, grass=1 has weight 0.0008
rain=0, sprinkler=1, grass=2 has weight 0.07840000000000001
rain=1, sprinkler=0, grass=0 has weight 0.0036000000000000003
rain=1, sprinkler=0, grass=1 has weight 0.014400000000000001
rain=1, sprinkler=0, grass=2 has weight 0.16200000000000003
rain=1, sprinkler=1, grass=0 has weight 0.0002
rain=1, sprinkler=1, grass=1 has weight 0.0008
rain=1, sprinkler=1, grass=2 has weight 0.019000000000000003


If random variable is not mentioned in the arguments to `value_product_indicators` then the result is the sum of the value product for each possible combination of states of the unmentioned random variables.

Warning: this is potentially computationally expensive if the state space of the unmentioned random variables is large.

In this example, `sprinkler` is not mentioned.

In [18]:
pgm.value_product_indicators(rain[1], grass[1])

0.015200000000000002

That is equivalent to summing over the states of `sprinker`.

In [19]:
sum (
    pgm.value_product_indicators(rain[1], grass[1], sprinkler[i])
    for i in sprinkler.state_range()
)

0.015200000000000002

If no indicators are mentioned, then this is equivalent to computing the partition function ($z$) of the PGM.

In [20]:
pgm.value_product_indicators()  # compute the value of the partition function

1.0000000000000002

## Compiling a PGM ##

To perform complex queries on a probabilistic graphical model, CK requires that it be compiled into an object that can be queried. The most common object is a `WMCProgram` object, which is constructed in two steps.

1. First the PGM is compiled into a circuit, using a PGM compiler, creating a `PGMCircuit` object.
2. Second, the circuit is compiled into a program, using a circuit compiler, with is usually wrapped into some inferencing object to enable probabilistic queries.

There are many different implementations of PGM compiler and circuit compiler.

The default PGM compiler is available as `ck.pgm_compiler.DEFAULT_PGM_COMPILER`.

Often, compilation of a circuit into a program is done automatically when an inferencing object is constructed. This will use the default circuit compiler, available as `ck.circuit_compiler.DEFAULT_CIRCUIT_COMPILER`.

In the following sample code, a `WMCProgram` object is created for probabilistic queries. It uses the default circuit compiler.

In [21]:
from ck.pgm_compiler import DEFAULT_PGM_COMPILER as pgm_compiler
from ck.pgm_circuit.wmc_program import WMCProgram
from ck.pgm_circuit import PGMCircuit

pgm_cct: PGMCircuit = pgm_compiler(pgm)  # Compile the PGM into an arithmetic circuit.

wmc = WMCProgram(pgm_cct)     # Make a WMCProgram object, using the default circuit compiler

Now efficient probability queries are possible. Consider this query using the original PGM...

In [22]:
pgm.value_product_indicators(rain[1], grass[1])

0.015200000000000002

Here is the equivalent query using a `WMCProgram`.

In [23]:
wmc.wmc(rain[1], grass[1])

0.015200000000000002

Here is the value of the parition function using the `WMCProgram`.

In [24]:
wmc.z

1.0

For a small PGM there may be little difference in computation time. The difference can be significant for a large PGM.

## Calculating probabilities ##

A `WMCProgram` provides many kinds of probabilistic query. Here are some examples.

What is the probability of 'not raining' and 'sprinkler off' and 'grass damp'?


In [25]:
wmc.probability(rain[0], sprinkler[0], grass[1])

0.0648

To make things clearer to read, we will define some states.

In [26]:
# for rain
not_raining = 0
raining = 1

# for sprinkler
off = 0
on = 1

# for grass
dry = 0
damp = 1
wet = 2

What is the probability of 'not raining' and 'sprinkler off' and 'grass damp'?

In [27]:
wmc.probability(rain[not_raining], sprinkler[off], grass[damp])

0.0648

Here is how we can calculate a marginal probability.

What is the marginal probability of 'grass damp'?

In [28]:
wmc.probability(grass[wet])

0.2666

We can calculate a marginal probability distribution.

What is the marginal probability distribution over sprinkler?

In [29]:
marginal = wmc.marginal_distribution(sprinkler)
print(marginal)

[0.9 0.1]


The result is a numpy array with probabilities co-indexed with the states of the given random variable.

Therefore, we can pretty-print the result if desired.

In [30]:
for state, pr in zip(sprinkler.states, marginal):
    print(pgm.indicator_str(sprinkler[state]), 'marginal probability is', pr)

sprinkler=0 marginal probability is 0.9
sprinkler=1 marginal probability is 0.1


## Conditional probabilities ##

Here is how you can calculate conditional probabilities with CK.

What is the probability distribution of sprinkler 'on', given 'not raining' and 'grass wet'?

A condition in CK is either an Indicator or a list/tuple of Indicators.


In [31]:
wmc.probability(sprinkler[on], condition=(rain[not_raining], grass[wet]))

0.9158878504672897

Semantically, the indicators of the given condition are grouped by random variable. The condition is interpreted as the conjunction of the groups, and the disjunction of the states within each group. For example, given condition for the Student PGM,
```
    grade('1'), intelligent('Yes'), grade(3)
```
then the condition interpretation is: (grade = 1 or grade = 3) and intelligent = Yes.

Marginal distributions can also be conditioned.

What is the marginal probability distribution over sprinkler, given 'not raining' and 'grass wet'?

In [32]:
marginal = wmc.marginal_distribution(sprinkler, condition=(rain[not_raining], grass[wet]))
print(marginal)

[0.08411215 0.91588785]


## Sampling a model ##

Samples may be drawn from a PGM using a sampler, created from a `WMCProgram`.

First create a sampler...

In [33]:
sampler = wmc.sample_direct()

A sampler provides an infinite iterator over instances.

In this example we limit ourselves to 8 samples.

In [34]:
for i, inst in enumerate(sampler, start=1):
    print(i, inst)
    if i == 8:
        break

1 [0, 0, 0]
2 [0, 1, 2]
3 [0, 0, 2]
4 [1, 0, 2]
5 [1, 0, 2]
6 [0, 0, 0]
7 [1, 0, 2]
8 [0, 0, 0]


A simple way to take a limited number of samples is to use `take`.

In [35]:
for inst in sampler.take(8):
    print(inst)

[0, 0, 0]
[0, 0, 0]
[1, 0, 2]
[0, 0, 0]
[0, 0, 0]
[1, 0, 2]
[0, 0, 0]
[1, 0, 1]


## MAP queries ##

We can perform MAP queries (maximum _a posteriori_), however, they are not efficient for large models with a `WMCProgram`. This is because the implementation typically performs an exhaustive search over the state space.

In [36]:
pr, states = wmc.map(grass, sprinkler)
print(f'MAP(grass, sprinkler) = {states}, with probability {pr}')

MAP(grass, sprinkler) = (0, 0), with probability 0.6516000000000001


## MPE queries ##

We can efficiently answer MPE queries using an `MPEProgram` object, also created from a `PGMCircuit` object. This can be practical even for large models.

In [37]:
from ck.pgm_circuit.mpe_program import MPEProgram

mpe_program = MPEProgram(pgm_cct)

mpe_result = mpe_program.mpe()
states = mpe_result.mpe
pr = mpe_result.wmc / wmc.z

print(f'MPE = {states}, with probability {pr}')


MPE = (0, 0, 0), with probability 0.6480000000000001


Here is the MPE, conditioned on grass = wet

In [38]:
condition = grass[wet]
mpe_result = mpe_program.mpe(condition)
states = mpe_result.mpe
pr = mpe_result.wmc / wmc.wmc(condition)

print(f'MPE(grass wet) = {states}, with probability {pr}')

MPE(grass wet) = (1, 0, 2), with probability 0.6076519129782447
